# 1. Importing Libraries including custom made

In [1]:
import os 
import subprocess
# Change the current working directory
os.chdir("C:\\Users\\Downloads\\BEM\\pcm_paper")

from pcm_class import IDF_EDITING
from pcm_block import MaterialProperty
from copy_to_path import copy_to_dir

# 2. Copy file from one directorty and added extension to name and copied to other directory

In [2]:
import os
# # Paths
# dir1 = r"C:\Users\Downloads\BEM\ASHRAE901_OfficeMedium_STD2013"
# dir2 = r"C:\Users\Downloads\BEM\ASHRAE_SO_2013\mediumPCM28"
# copy_to_dir(dir1, dir2, file_type='idf', extension='_pcm28.')

#  Testing with one file
dir1 = r"C:\Users\Downloads\BEM\pcm_paper\pcm_idf"
dir2 = r"C:\Users\Downloads\BEM\pcm_paper\pcm_idf\new"
copy_to_dir(dir1, dir2,file_type='idf', extension='_pcm28.')

# 3. Finding idf file names in the directory

In [3]:
## List of idf in a folder
import os
def idf_file_list(file_path):
    # Import necessary libraries
    import os, glob, shutil
    # Change directory to old idf files to work with
    os.chdir(file_path)

    # Getting names of all idf files only
    files = glob.glob("*.idf")
    return files

file_path = r"C:\Users\Downloads\BEM\pcm_paper\pcm_idf\new"
idf_files = idf_file_list(file_path)
print(idf_files)

['ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28.idf', 'ASHRAE901_OfficeMedium_STD2013_Rochester_pcm28.idf']


In [4]:
## Find the name of wall/roof/floor
idf_file = idf_files[0] # choose just any one of the idf file

def find_envelope_name(file_path):
    lines = []
    with open(file_path, 'r') as rf:
        lines = rf.readlines()

    lists = lines.copy()

    # Create a dictionary with line numbers and contents
    words = {}
    for idx, item in enumerate(lists):
        words[idx] = item.strip()

    # Prepare lists to store wall, roof, and floor names
    wall_names = []
    roof_names = []
    floor_names = []

    # Iterate through the lines to find BuildingSurface:Detailed objects
    for key, val in words.items():
        if 'BuildingSurface:Detailed' in val:
            current_key = key
            # Extract the surface name and exclude anything after '!'
            surface_name = words[current_key + 3].split(',')[0].strip()

            if "Wall" in words[current_key + 2]:
                wall_names.append(surface_name)
            elif "Roof" in words[current_key + 2]:
                roof_names.append(surface_name)
            elif "Floor" in words[current_key + 2]:
                floor_names.append(surface_name)

    return wall_names, roof_names, floor_names

# Example usage
file_path = rf"C:\Users\Downloads\BEM\pcm_paper\pcm_idf\new\{idf_file}"
wall_names, roof_names, floor_names = find_envelope_name(file_path)

## set ensure one name appeared only once
print("Walls:", set(wall_names))
print("Roofs:", set(roof_names))
print("Floors:", set(floor_names))

Walls: {'nonres_ext_wall_grd', 'nonres_ext_wall_mid', 'int_wall', 'nonres_ext_wall_top'}
Roofs: {'nonres_roof'}
Floors: {'Perimeter_bot_ZN_2_Floor_Ffactor', 'Perimeter_bot_ZN_4_Floor_Ffactor', 'Perimeter_bot_ZN_3_Floor_Ffactor', 'int_slab_floor', 'Core_bot_ZN_5_Floor_Ffactor', 'Perimeter_bot_ZN_1_Floor_Ffactor'}


# 4. User input for material proprties 

In [5]:
# List of envelope names (walls, roofs, and floors) you want to edit
walls = ['nonres_ext_wall_top', 'nonres_ext_wall_grd', 'nonres_ext_wall_mid']
roofs = ['nonres_roof']
floors = ['Core_bot_ZN_5_Floor_Ffactor']

# Choose the envelope names you want to edit (can be a single item or a list of surfaces)
_envelope_names = walls  # You can use 'walls', 'roofs', 'floors', or a combination like ['nonres_roof', 'nonres_ext_wall_top']

_split_word = 'Layer ' # Where PCM layer will be added
_pcm_layer = "!- Layer 1"

# Example PCM to use
pcm_mat = MaterialProperty('pcmboard')

# PCM23
# pcm_mat.material(0.025, 0.27, 1000, 2120)
# pcm_mat.phasechange(-20.0, 0, 23, 55000, 24, 75000, 100, 165000)

# PCM28
pcm_mat.material(0.025, 0.25, 814, 2150)
pcm_mat.phasechange(-20.0, 0, 28, 103200, 28.1, 226300, 100, 481000)
_pcm_block = pcm_mat.pcmblock()

# 5. Editing idf files with PCM properties

In [ ]:
# Loop through each IDF file
for file_name in idf_files:
    file01 = os.path.abspath(file_name)  # Get absolute path of IDF file
    file_path = file01

    count = 0 # counting number of enveoples to which PCM is added
    for _envelope_name in _envelope_names:
        # Create an instance of IDF_EDITING for each surface
        file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer, _pcm_block)
        
        # Editing the entire IDF file with PCM properties
        if count<=0:
            file_object.create_idf()
            
        else:
        # It will only create idf without appending PCM properties
        # For PCM addition to more than one envelope, pcm properties need to include once in IDF.
            file_object.create_idf__no_appending()

        count+=1

    print(f"Finished processing {file_name}")

# ## Testing with one file only so loop is commented out  
# file01 = os.path.abspath(idf_files[1])
# file_object = IDF_EDITING(file01, _envelope_name, _split_word, _pcm_layer,  _pcm_block)
# file_object.create_idf()

Finished processing ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28.idf
Finished processing ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28.idf
Finished processing ASHRAE901_OfficeMedium_STD2013_Rochester_pcm28.idf


# 6. Creating new directories for each idf file and run simulation 

In [ ]:
import os
import subprocess

# List of simulation results' directory
output_directory_list = []

def find_matching_epw(epw_directory, idf_filename):
    base_name = idf_filename.replace("_pcm28", "").split('_')[-1].split('.')[0]
    # base_name = idf_filename.replace("_pcm23", "").split('_')[-1].split('.')[0]
    
    for epw_file in os.listdir(epw_directory):
        if epw_file.endswith('.epw') and base_name.lower() in epw_file.lower():
            return os.path.join(epw_directory, epw_file)
    return None

def run_energyplus(idf_directory, epw_directory, eplus_dir):
    for file_name in os.listdir(idf_directory):
        if file_name.endswith('.idf'):
            idf_filepath = os.path.join(idf_directory, file_name)
            base_name = os.path.splitext(file_name)[0]
            output_directory = os.path.join(idf_directory, f"results_{base_name}")

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            output_directory_list.append(output_directory) 
            
            matched_epw = find_matching_epw(epw_directory, file_name)
            if matched_epw:
                cl_st = [
                    f"{eplus_dir}\\EnergyPlus",
                    "--readvars",  # included to create a .csv file of the results
                    f"--output-directory={output_directory}",
                    f"--weather={matched_epw}",
                    idf_filepath,
                ]

                result = subprocess.run(cl_st, capture_output=True, text=True)

                if result.returncode == 0:
                    print(f"Simulation for {file_name} completed successfully.")
                else:
                    print(f"Simulation for {file_name} failed.")
                    print("Error:", result.stderr)
            else:
                print(f"No matching .epw file found for {file_name}.")

# idf_directory = r'C:\Users\Downloads\BEM\ASHRAE_SO_2013\pcm28'  # Path to .idf files
# epw_directory = r'C:\Users\Downloads\BEM\ASHRAE901_epw'  # Path to .epw files

idf_directory = r'C:\Users\Downloads\BEM\pcm_paper\pcm_idf\new'  # Path to .idf files
epw_directory = r'C:\Users\Downloads\BEM\ASHRAE901_epw'  # Path to .epw files
eplus_dir = r'C:\EnergyPlusV22-1-0'  # Path to EnergyPlus installation
 
run_energyplus(idf_directory, epw_directory, eplus_dir)

Simulation for ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28.idf completed successfully.
Simulation for ASHRAE901_OfficeMedium_STD2013_Rochester_pcm28.idf completed successfully.


# 7. Getting results

In [22]:
## Printing the new created folders with simulation results 
print(output_directory_list)

['C:\\Users\\Downloads\\BEM\\pcm_paper\\pcm_idf\\new\\results_ASHRAE901_OfficeMedium_STD2013_Fairbanks_pcm28', 'C:\\Users\\Downloads\\BEM\\pcm_paper\\pcm_idf\\new\\results_ASHRAE901_OfficeMedium_STD2013_InternationalFalls_pcm28', 'C:\\Users\\Jayedi Aman\\Downloads\\BEM\\pcm_paper\\pcm_idf\\new\\results_ASHRAE901_OfficeMedium_STD2013_Rochester_pcm28']


In [25]:
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup

def htm_file_list(directory):
    """Returns a list of .htm files in the given directory."""
    if not os.path.isdir(directory):
        raise NotADirectoryError(f"{directory} is not a valid directory.")
    return glob.glob(os.path.join(directory, "*.htm"))


def extract_heating_cooling_loads(html_file):
    """Extracts total heating and cooling load from an EnergyPlus HTML report."""
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Locate "End Uses" table
    end_uses_tag = soup.find('b', string="End Uses")
    if not end_uses_tag:
        return None, None  # No table found, return None values

    target_table = end_uses_tag.find_next('table')
    if not target_table:
        return None, None  # No table found

    # Initialize load values
    heating_load = 0.0
    cooling_load = 0.0
    fan_load = 0.0

    # Process table rows
    rows = target_table.find_all('tr')[1:]  # Skip header row
    for row in rows:
        cols = row.find_all('td')
        end_use = cols[0].text.strip().lower()

        if end_use == "heating":
            heating_load = sum(float(col.text.strip()) for col in cols[1:])
        elif end_use == "cooling":
            cooling_load = sum(float(col.text.strip()) for col in cols[1:])
        elif end_use == "fans":
            fan_load = sum(float(col.text.strip()) for col in cols[1:])


    return heating_load, cooling_load, fan_load

# Initialize DataFrame
df_results = pd.DataFrame(columns=['Folder Name', 'File Name', 'Heating Load (GJ)', 'Cooling Load (GJ)', 'Fan Load (GJ)', 'Total HVAC Load (GJ)'])

# Process all output directories
for folder in output_directory_list:
    folder = os.path.abspath(folder)  # Ensure absolute path
    try:
        htm_files = htm_file_list(folder)
        folder_name = os.path.basename(folder)  # Extract the last directory name

        for htm_file in htm_files:

            heating, cooling, fan = extract_heating_cooling_loads(htm_file)
            total = float(heating) + float(cooling) + float(fan)

            if heating is not None and cooling is not None:
                new_row = pd.DataFrame([[folder_name, os.path.basename(htm_file), heating, cooling, fan, total]], 
                       columns=['Folder Name', 'File Name', 'Heating Load (GJ)', 'Cooling Load (GJ)', 'Fan Load (GJ)', 'Total HVAC Load (GJ)'])

                if not new_row.isna().all().all():  # Check if the new row is not all NaN
                    df_results = pd.concat([df_results, new_row], ignore_index=True)

    except NotADirectoryError as e:
        print(e)

# Save results to a CSV file
output_csv = os.path.join(os.getcwd(), "heating_cooling_loads.csv")
df_results.to_csv(output_csv, index=False)

# Display final DataFrame
print(df_results)


C:\Users\AppData\Local\Temp\ipykernel_1264\534154968.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row], ignore_index=True)


                                         Folder Name     File Name  \
0  results_ASHRAE901_OfficeMedium_STD2013_Fairban...  eplustbl.htm   
1  results_ASHRAE901_OfficeMedium_STD2013_Interna...  eplustbl.htm   
2  results_ASHRAE901_OfficeMedium_STD2013_Rochest...  eplustbl.htm   

   Heating Load (GJ)  Cooling Load (GJ)  Fan Load (GJ)  Total HVAC Load (GJ)  
0             855.22              51.84          95.09               1002.15  
1             503.93             121.02          98.86                723.81  
2             681.99             160.72          98.89                941.60  


# F8 Metal deleted from Medium Office